# Exp4: 基于K-近邻的车牌号识别

## 一、案例简介

图像的智能处理一直是人工智能领域广受关注的一类技术，代表性的如人脸识别与 CT 肿瘤识别，在人工智能落地的进程中发挥着重要作用。其中车牌号识别作为一个早期应用场景，已经融入日常生活中，为我们提供了诸多便利，在各地的停车场和出入口都能看到它的身影。车牌号识别往往分为字符划分和字符识别两个子任务，本案例我们将关注字符识别的任务，尝试用 K-NN 的方法对分割好的字符图像进行自动识别和转化。

## 二、作业说明

### 基本要求
* 完成数据的读入和表示，将图片表示成向量并和 label 对应上；
* 构建 K-NN 模型（可调库）对测试集中的图片进行预测并计算准确率；
* 分析当 K 取不同值时测试准确率的变化。

### 扩展要求
* 分析不同距离度量方式对模型效果的影响；
* 对比平权和加权 K-NN 的效果；
* 分析训练集大小对测试结果的影响。

## 三、数据概览
本次我们使用已经分割好的车牌图片作为数据集，包括数字 0-9、字母 A-Z（不包含 O 和 I）以及省份简称共 65 个类，编号从 0 到 64。数据已经分成了训练集和测试集，里面的文件夹用 label 编号命名，一个文件夹下的所有图片都属于该文件夹对应的类，每个图片都是 20 * 20 的二值化灰度图。

下面演示一下如何借助 PIL 库将图片转化为向量：

In [68]:
from PIL import Image
import numpy as np
from pathlib import Path
from typing import List,Dict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [69]:

img = Image.open('data/train/0/4-3.jpg')  # 打开图片
pixels = np.array(img)  # 转化为 numpy 矩阵
print(pixels.shape)
img  # 显示图片from PIL import Image

(20, 20)


In [70]:
# tools for load train and test datas
def GlobFolder(folder:Path):
    subfolders = []
    for sub in folder.iterdir():
        if sub.is_dir():
            subfolders.append(sub)
    return subfolders

def GlobJPGFile(folder:Path):
    return list(folder.glob("*.jpg"))

def LoadLableAndDatas(root:Path):
    # input : std::filesystem::path . means folder contain
    # output: std::map<int , cv::Mat>
    lable_imgs = {}
    total_sub_folder:List[Path] = GlobFolder(root)
    for each_folder in total_sub_folder:
        total_img_file = GlobJPGFile(each_folder)
        lable = int(each_folder.stem)
        lable_imgs[lable]=[]
        for each_jpg_file in total_img_file:
            img = Image.open(each_jpg_file.as_posix())
            # Important: Normalize grayscale values to the [0, 1] range
            pixels = np.array(img, dtype=np.float32)
            pixels /= 255
            pixels = pixels.flatten()
            lable_imgs[lable].append(pixels)
    return lable_imgs

def SeparateLableAndDatas(data):
    # input: std::map<int , std::vector<cv::Mat> >  // means lable and vector of img array
    # output: std::vector<int> , std::vector<cv::Mat> // vector of lable and vector of img array
    lables = []
    datas  = []
    for lable , total_array in data.items():
        for array in total_array:
            lables.append(lable)
            datas.append(array)
    return lables, datas
    

In [71]:

# Load datas
cwd = Path.cwd()
train_folder = cwd/('data/train')
test_folder = cwd/('data/test')
if not train_folder.exists() or not test_folder.exists():
    print(f"Error with load datas {train_folder.as_posix()} , {test_folder.as_posix()}")
    exit(1)
# datas type std::map<int , std::vector<cv::Mat> >
train_lable_imgs = LoadLableAndDatas(train_folder)
test_lable_imgs = LoadLableAndDatas(test_folder)

train_lables, train_arrays = SeparateLableAndDatas(train_lable_imgs)
test_lables , test_arrays = SeparateLableAndDatas(test_lable_imgs)
assert( len(train_lables)== len(train_arrays) and len(test_lables) == len(test_arrays))
print(f"Load train/test lable size {len(train_lable_imgs)} / {len(test_lable_imgs)}")
print(f"Load train/test data  size {len(train_arrays)} / {len(test_arrays)}")

Load train/test lable size 65 / 65
Load train/test data  size 15954 / 4665


## 四、模型构建
已经读取 归一化后 的 向量格式数据

- train_lables: 训练集标签 
- train_arrays: 训练集数据
- test_lables : 测试集标签
- test_arrays : 测试集数据

In [ ]:
# 尝试一下GridSearchCV,上次作业没用上
param_grid = {
    'n_neighbors': [1, 3, 5, 7],
    'weights': ['uniform', 'distance'], # 距离计算方法，uniform 等权重，distance 根据距离设置权重投票
    'p': [1, 2]                         # 距离度量：1=曼哈顿，2=欧氏
}

grid_search = GridSearchCV(
    KNeighborsClassifier(), 
    param_grid, 
    scoring='accuracy',
    verbose=1,
    cv=5)

grid_search.fit(train_arrays, train_lables)
print(f"Best params: {grid_search.best_params_}")
best_model = grid_search.best_estimator_
print(f"Best acc: {best_model.score(test_arrays, test_lables)}")

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best params: {'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
Best acc: 0.7168274383708467


In [74]:
nlist = [1,3,5,7]
wlist = ['uniform','distance']
plist = [1,2]
for n in nlist:
    for w in wlist:
        for p in plist:
            knn = KNeighborsClassifier(
                n_neighbors=n,
                weights= w,         # uniform , distance
                algorithm='auto',   # auto , ball_tree , kd_tree , brute
                p=p,                # p=2 
                n_jobs=-1           # for cpu core
                )
            knn.fit(train_arrays , train_lables)

            lable_pred = knn.predict(test_arrays)
            accuracy = accuracy_score(lable_pred , test_lables)
            print(f" Using (K ={n} with {"uniform " if w == 'uniform' else 'distance'} weights and {"Euclidean" if p==2 else "Manhattan"} distance). Accuracy socre: {accuracy}")
    print("")

 Using (K =1 with uniform  weights and Manhattan distance). Accuracy socre: 0.7142550911039657
 Using (K =1 with uniform  weights and Euclidean distance). Accuracy socre: 0.7168274383708467
 Using (K =1 with distance weights and Manhattan distance). Accuracy socre: 0.7142550911039657
 Using (K =1 with distance weights and Euclidean distance). Accuracy socre: 0.7168274383708467

 Using (K =3 with uniform  weights and Manhattan distance). Accuracy socre: 0.6906752411575563
 Using (K =3 with uniform  weights and Euclidean distance). Accuracy socre: 0.69989281886388
 Using (K =3 with distance weights and Manhattan distance). Accuracy socre: 0.7061093247588425
 Using (K =3 with distance weights and Euclidean distance). Accuracy socre: 0.7103965702036441

 Using (K =5 with uniform  weights and Manhattan distance). Accuracy socre: 0.6827438370846731
 Using (K =5 with uniform  weights and Euclidean distance). Accuracy socre: 0.6928188638799572
 Using (K =5 with distance weights and Manhattan d

## 总结
**基本要求**： 
- 数据读取中 将标签设置为文件夹名称，图片已经归一化并转换成向量，两者一一对应。
- 构建KNN模型，计算准确率 max:0.7168274383708467。 
- 尝试了K分别取1,3,5,7,9,发现在K=1时 准确率最高，也就是之用一个邻居来投票效果最好，同时还发现在k=1时，等权重和带权重的方法准确率一样，这个结论也复合预期，因为只有1个邻居投票，占了全部的权重。当k依次曾大时，准确率降低，说明训练出来的模型中某些特征取值的附近空间中存在其他类别的特征，也就是不同标签的邻居干扰了投票结果。
**扩展要求**：
- 根据不同参数对比，发现使用Euclidean距离 的准确率 要优于Manhattan距离。借助GPT：欧式距离（Euclidean）往往比曼哈顿距离（Manhattan）效果更好，这是因为它更能捕捉图像的整体几何形状差异。而曼哈顿距离是 L1 范数，对所有像素差异一视同仁（线性相加）。它更适合用于稀疏、高维、非结构化数据（如词袋模型、推荐系统），而不是具有空间结构的图像。
- 根据不同参数对比，发现加权要比平权准确率更高，这说明 在本次任务中 加权的权重策略使得 分类边界和抗干扰能力更好。
**未实现**：
分析训练集大小对测试结果的影响